<a href="https://colab.research.google.com/github/Ashish-17CSE/Hate_Speech_Detection/blob/main/SVM_for_Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload Data


In [ ]:
# Upload the train.csv and val.csv file from your local drive
from google.colab import files
uploaded = files.upload()

Saving dev_3k_split_conll.txt to dev_3k_split_conll.txt
Saving Hindi_test_unalbelled_conll_updated.txt to Hindi_test_unalbelled_conll_updated.txt
Saving test_labels_hinglish.txt to test_labels_hinglish.txt
Saving train_14k_split_conll .txt to train_14k_split_conll .txt


In [ ]:
dataset = '/content/dev_3k_split_conll.txt'

def read_conll(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words, lang = [], []
        id = "";
        label = "";
        for line in f.readlines():
            sp = line.strip().split('\t')
            if sp[0]=="meta" and len(sp)>2:
              vc = {"id":id,"label":label,"words":words,"lang":lang}
              data.append(vc);
              words,lang = [],[]
              id = sp[1]
              label = sp[2]
            elif len(sp)>1:
              words.append(sp[0])
              lang.append(sp[1])
        vc = {"id":id,"label":label,"words":words,"lang":lang}
        data.append(vc);
    return data 

MyData = read_conll(dataset)[1:];
print(MyData)

In [ ]:
import pandas as pd

In [ ]:
def read_conll(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words, lang = [], []
        id = "";
        label = "";
        for line in f.readlines():
            sp = line.strip().split('\t')
            if sp[0]=="meta" and len(sp)>2:
              vc = {"id":id,"label":label,"words":words,"lang":lang}
              data.append(vc);
              words,lang = [],[]
              id = sp[1]
              label = sp[2]
            elif len(sp)>1:
              words.append(sp[0])
              lang.append(sp[1])
        vc = {"id":id,"label":label,"words":words,"lang":lang}
        data.append(vc);
    return data 


In [ ]:
#convert to sentances

def read_conll(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words = " "
        id = "";
        label = "";
        for line in f.readlines():
            sp = line.strip().split('\t')
            if sp[0]=="meta" and len(sp)>2:
              vc = {"id":id,"label":label,"tweet":words}
              data.append(vc);
              words = " "
              id = sp[1]
              label = sp[2]
            elif len(sp)>1:
              if sp[1]!="O":
                words = words + sp[0] +" " #+"("+sp[1]+") "
        vc = {"id":id,"label":label,"tweet":words}
        data.append(vc);
    return data 

In [ ]:
#Merge test labels and tweets
'''
 Me Student hu
 Hin Eng     Hin

 Me Student Hu(Hin)
'''

def read_conll_test(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words = " "
        id = "";
        label = "";
        for line in f.readlines():
            print(line)
            sp = line.strip().split('\t')
            if sp[0]=="meta":
              vc = {"id":id,"tweet":words}
              data.append(vc);
              words = " "
              id = sp[1]
            elif len(sp)>1:
              if sp[1]!="O":
                words = words + sp[0] +" " # +"("+sp[1]+") "
        vc = {"id":id,"tweet":words}
        data.append(vc);
    return data 

dataset = "/content/Hindi_test_unalbelled_conll_updated.txt" #"/content/test.txt"
test = read_conll_test(dataset)[1:]
test = pd.DataFrame(test)
test.head()

In [ ]:
labels = pd.read_csv("/content/test_labels_hinglish.txt") #("/content/test_labels.txt")
labels = pd.DataFrame(labels)
labels.columns=["id","label"] 
labels['id']=labels['id'].astype(str)
labels.head()

,id,label
0,20803,neutral
1,20187,neutral
2,20953,neutral
3,13777,negative
4,20990,positive


In [ ]:
print(test.dtypes)
print(labels.dtypes)

id       object
tweet    object
dtype: object
id       object
label    object
dtype: object


In [ ]:
# test = pd.concat([test, labels], axis = 1)
test = pd.merge(test, labels, how='inner', on = 'id')
test.head()

,id,tweet,label
0,20803,454dkhan Heisunberg Agr kse ko itni important...,neutral
1,20187,logon ko alloo pyaz tomator me toh allah pak ...,neutral
2,20953,LambaAlka Wafadaar bane rahane ka nayab tarik...,neutral
3,13777,varnishant narendramodi Chup bhosdike He has ...,negative
4,20990,RT HardeepSPuri Praise the Great Giver who gi...,positive


In [ ]:
import pandas as pd
dataset1 = '/content/train_14k_split_conll .txt'#'/content/train.txt'
dataset2 = '/content/dev_3k_split_conll.txt'#'/content/valid.txt'

train = read_conll(dataset1)[1:];
valid = read_conll(dataset2)[1:];
train = pd.DataFrame(train)
valid = pd.DataFrame(valid)
# print(train.head())
valid.head()

,id,label,tweet
0,30258,positive,prahladspatel modi mantrimandal may samil hon...
1,16648,negative,bkunalraj TajinderBagga NikhilJakhar14 JammuP...
2,28511,negative,waglenikhil U saw caste and religion in them ...
3,10466,neutral,DelhiPolice sir local police station pe compl...
4,19266,positive,Ve Maahi song from Kesari is current favourit...


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import GaussianNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

from nltk.corpus import stopwords
import re
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# !pip install sastrawi

In [ ]:
train['label'].value_counts()

neutral     5264
positive    4634
negative    4102
Name: label, dtype: int64

In [ ]:
train.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [ ]:
train.isnull().values.any()

False

## Data Preprocessing

Data Cleaning

In [ ]:
train['cleaned_tweet'] = train.tweet.apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('@')]))

In [ ]:
train['cleaned_tweet'] = train.tweet.apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('@')]))
train['cleaned_tweet'] = train.tweet.apply(lambda x: ' '.join([word for word in x.split() if not word.endswith(':')]))

In [ ]:
train_clean = train.copy()
train_clean.head()

In [ ]:
# import nltk
# nltk.download("stopwords")
from nltk.corpus import stopwords
def remove_stopwords(text):
  words = [w for w in text if w not in stopwords.words('english')]
  return words

train['tweet'] = train['tweet'].str.lower()
valid['tweet'] = valid['tweet'].str.lower()
test['tweet'] = test['tweet'].str.lower()


train['tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
valid['tweet'] = valid['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
test['tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

train['tweet'] = train['tweet'].apply(lambda x: remove_stopwords(x))
valid['tweet'] = valid['tweet'].apply(lambda x: remove_stopwords(x))
test['tweet']  = test['tweet'].apply(lambda x: remove_stopwords(x))

In [ ]:
def tweet_processing(raw_tweet):
    letters_only=re.sub("[^a-zA-Z]"," ",raw_tweet)
    words=letters_only.lower().split()
    stops=set(stopwords.words("english"))
    m_w=[w for w in words if not w in stops]
    return (" ".join(m_w))

In [ ]:
num_tweets=train["tweet"].size
clean_tweet=[]
for i in range(0,num_tweets):
    clean_tweet.append(tweet_processing(train["tweet"][i]))
train["tweet"]=clean_tweet 


num_tweets_test=test["tweet"].size
clean_tweet_test=[]
for i in range(num_tweets,num_tweets+num_tweets_test):
    clean_tweet_test.append(tweet_processing(test["tweet"][i]))
test["tweet"]=clean_tweet_test

## Vectorization

In [ ]:
X = train_clean['cleaned_tweet']
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)
X_train_counts.shape

(14000, 47205)

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(14000, 47205)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(train_clean['label'])
y

array([1, 1, 0, ..., 2, 2, 1])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=0)

## Classifier

## Baseline model : SVM

In [ ]:
X_train, X_test_svm, Y_train, Y_test_svm = train_test_split(train.tweet, train.label, test_size=0.2)

In [ ]:
vectorizer=CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None,max_features = 5000)

train_data_features=vectorizer.fit_transform(X_train)
train_data_features=train_data_features.toarray()

test_data_features=vectorizer.transform(X_test_svm)
test_data_features=test_data_features.toarray()

In [ ]:
#SVM with linear kernel
clf=svm.SVC(kernel='linear',C=1.0)
print ("Training")
clf.fit(train_data_features,Y_train)

print ("Testing")
predicted=clf.predict(test_data_features)
accuracy=np.mean(predicted==Y_test_svm)
print ("Accuracy: ",accuracy)

Training
Testing
Accuracy:  0.5410714285714285
